# 01 Quickstart / クイックスタート

This notebook demonstrates the end-to-end flow on a synthetic tire. It mirrors the CLI pipeline: load points, fit a cylinder, pick (synthetic) rim points, slice the 12 o'clock band, and output an X–Z' profile.

In [ ]:
# Ensure the repository root is importable (VS Code / Windows friendly)
import sys
from pathlib import Path

NOTEBOOK_ROOT = Path().resolve()
repo_root = None
for candidate in [NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, NOTEBOOK_ROOT.parent.parent]:
    if (candidate / "tire_profiler").exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError("Unable to locate tire_profiler package relative to this notebook")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tire_profiler.align import align_points
from tire_profiler.cylinder import fit_cylinder
from tire_profiler.rimline import fit_rimline
from tire_profiler.slice_profile import cylindrical_features, slice_band, compute_profile, plot_profile

In [ ]:
def synthetic_tire_points(seed: int = 0, n: int = 120_000):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-0.12, 0.12, n)
    theta = rng.uniform(-np.pi, np.pi, n)
    radius = 0.34 + 0.002 * np.sin(6 * theta)
    y = radius * np.sin(theta)
    z = radius * np.cos(theta)
    rim_mask = np.abs(x) > 0.09
    z[rim_mask] += 0.006 * np.exp(-((theta[rim_mask]) ** 2) / 0.1)
    noise = rng.normal(scale=0.0008, size=(n, 3))
    pts = np.stack([x, y, z], axis=1) + noise
    return pts

In [ ]:
points = synthetic_tire_points(seed=42)
points.shape

In [ ]:
model = fit_cylinder(points, threshold=0.0025)
aligned, *_ = align_points(points, model.point_on_axis, model.axis_direction)
features = cylindrical_features(aligned, model.radius)
mask = slice_band(aligned, features=features, tape_width=0.02, outer_band=0.05)
print(f"Selected {mask.sum()} points in the 12 o'clock band")

In [ ]:
# Synthetic rim picks near the side wall so the notebook can run non-interactively
side_mask = (np.abs(aligned[:, 0]) > 0.11) & (np.abs(features['arc']) < 0.03)
rim_points = aligned[side_mask][:20]
arc = np.arctan2(rim_points[:, 1], rim_points[:, 2]) * model.radius
rimline = fit_rimline(rim_points, arc)
rimline

In [ ]:
result = compute_profile(
    aligned,
    features=features,
    mask=mask,
    rimline=rimline,
    nbins=80
)
result.profile.head()

In [ ]:
plot_profile(result.profile, Path('quickstart_profile.png'))
result.profile[['x_center', 'z_mean', 'z_std']].plot(x='x_center', y='z_mean', kind='line')
plt.title("Preview of Z' mean")
plt.show()